# VIs Calculation Using HLS Imagery

Using the Harmonized Landsat and Sentinel-2 (HLS) surface reflectance dataset<sup> [1]</sup>, the team is developing canopy chlorophyll and Gross Primary Production (GPP) equations based on Vegetation Indices (VIs). The team was working with extracted HLS data and VIs, and is now applying the equations to the HLS imagery. The data for this project is currently available under the NASA Center for Climate Simulation (NCCS) ADAPT system, primarily located in the following directories:

```bash
L30: /att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30
S30: /att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.S30
YEARS: 2015-2019
```

This notebook is an enhancement of existing MATLAB scripts in order to calculate VIs from extracted spectral values. The idea is to apply the calculated indices to the images and make spatial/geo-referenced maps of the VIs. The team would like to scale the indices 0-1 and apply the canopy chlorophyll and GPP equations to the VI images (scaled and not scaled). This notebook is arquitected to apply to L30 and S30 imagery, and it will require slight modifications in order to apply to additional datasets.

**Author:** Jordan A. Caraballo-Vega - NASA GSFC, <jordan.a.caraballo-vega@nasa.gov> <br/>
**Release Date:** 07/01/2021 <br/>
**Version:** 1.0.1 <br/>

TODO:

- MASK pixels if outside of the image or NaN, else %in image – I was extracting the spectra ( I think that I am not doing this)
- Remove original bands from the resulting geotif?
- Create environment file


## 1. Usage and installation requirements

## 1.1 Creating a conda environment - TODO

In order to run this notebook you will need a conda environment with all dependencies installed. ADAPT provides a built-in environment from the JupyterHub interface that is only missing a couple of packages that can be installed on the fly. In order to get started quickly, follow the next steps:

Login to adaptlogin
module load anaconda
conda env create from yaml file from here


Same steps can be done from your laptop

1. Login to the NCCS JupyterHub <https://www-proxy-dev.nccs.nasa.gov/jupyterhub-adapt/>.
2. Open this notebook via the file/upload method.
3. Select kernel, in this case "earthml".
4. Execute the cell listed below (press the Run button).

## 1.2 Login to ADAPT JupyterHub

To leverage NCCS ADAPT resources, you will need to login to ADAPT JupyterHub. The steps are outlined below.

1. Login to the NCCS JupyterHub <https://www-proxy-dev.nccs.nasa.gov/jupyterhub-adapt/>.
2. Open this notebook via the file/upload method.
3. Select kernel, in this case "hlsgpp".
4. Start working on your notebook.

## 2. Define global variables for the HLS dataset


## 2.1 Import Python Libraries

In this section we include all Python libraries required to execute the code below. There are no external code dependencies besides the packages installed under section 1.1.

In [1]:
import os
from glob import glob
import numpy as np
import rioxarray as xrx
import matplotlib.pyplot as plt
from matplotlib import colors
import ipywidgets as widgets
import multiprocessing as mp

print("Number of processors: ", mp.cpu_count())

Number of processors:  12


## 2.1 Define Global Variables

In this section we define global variables utilized across the entire notebook. The variables together with their description are listed below.

- **MASK_OPTIONS (dict):** The HLS dataset provides a set of masks that can be retrieved from the QA band of the imagery. The variable MASK_OPTIONS lets the user define which masks are available for selection. This variable will then be used in the checkbox menu to determine which masks will be calculated and applied.
- **VIS_OPTIONS_S30 (dict):** A list of VIs is available through the provided MATLAB script. Different sets of VIs can be calculated based on the selected satellite. This variable defines the available VIs from the Sentinel imagery.
- **VIS_OPTIONS_L30 (dict):** Similar to VIS_OPTIONS_S30, this varible defines the available VIs from the Landsat imagery.

No input from the user is required in this section, unless bands in the imagery change, or new VIs are introduced in this project for calculation.

In [2]:
# DO NOT MODIFY THIS SECTION UNLESS YOU HAVE MORE MASKS OR INDICES TO INCLUDE
# IPYWIDGETS REQUIRES A DICTIONARY THAT INCLUDES BOTH THE NAME AND LABEL

# MASK_OPTIONS: DICTIONARY WITH AVAILABLE MASKS
MASK_OPTIONS = {
    'ALL':         'ALL',
    'ADJCLOUD':    'ADJCLOUD',
    'AQ':          'AQ',
    'CIRRUS':      'CIRRUS',
    'CLOUD':       'CLOUD',
    'CLOUDSHADOW': 'CLOUDSHADOW',
    'SNOW':        'SNOW',
    'WATER':       'WATER'
}

In [3]:
# VIS_OPTIONS_S30: DICTIONARY WITH AVAILABLE VIS FOR S30
VIS_OPTIONS_S30 = {
    'ALL':         'ALL',
    'CI705':       'CI705',
    'CI740':       'CI740',
    'CCCI':        'CCCI',
    'CIG':         'CIG',
    'CIRE':        'CIRE',
    'CVI':         'CVI',
    'EVI':         'EVI',
    'FCVI':        'FCVI',
    'FCVI_VIS':    'FCVI_VIS',
    'GLI':         'GLI',
    'GNDVI':       'GNDVI',
    'MCARI':       'MCARI',
    'MCARI_MTVI2': 'MCARI_MTVI2',
    'MSAVI':       'MSAVI',
    'MTCI':        'MTCI',
    'MTVI1':       'MTVI1',
    'MTVI2':       'MTVI2',
    'NDREI':       'NDREI',
    'NDVI':        'NDVI',
    'NDVI705':     'NDVI705',
    'NDVI740':     'NDVI740',
    'NGRDI':       'NGRDI',
    'OSAVI':       'OSAVI',
    'RE1RE2':      'RE1RE2',
    'REIP3':       'REIP3',
    'SAVI':        'SAVI',
    'SR':          'SR',
    'TCARI':       'TCARI',
    'TCARI_OSAVI': 'TCARI_OSAVI',
    'TCI':         'TCI',
    'TGI':         'TGI',
    'TVI':         'TVI',
    'VARI':        'VARI'
}

# VIS_OPTIONS_L30: DICTIONARY WITH AVAILABLE VIS FOR L30
VIS_OPTIONS_L30 = {
    'ALL':         'ALL',
    'CIG':         'CIG',
    'CVI':         'CVI',
    'EVI':         'EVI',
    'FCVI':        'FCVI',
    'FCVI_VIS':    'FCVI_VIS',
    'GLI':         'GLI',
    'GNDVI':       'GNDVI',
    'MSAVI':       'MSAVI',
    'MTVI1':       'MTVI1',
    'MTVI2':       'MTVI2',
    'NDVI':        'NDVI',
    'NGRDI':       'NGRDI',
    'OSAVI':       'OSAVI',
    'SAVI':        'SAVI',
    'SR':          'SR',
    'TVI':         'TVI',
    'VARI':        'VARI'
}

## 3. Define variables to extract VIs from the HLS imagery

## 3.1 Widget Functions

In this section we define widget functions to assist in the execution of this notebook. These functions take care of the visual implementations of elements that will then be used to finalize Mask and VIs calculations. No input from the user is required in this section.

In [4]:
def checkbox_menu(data: dict) -> list:
    """
    Define dynamic widgets for checkbox menu.
    Input:
        data (dict): dictionary with key and label to initiate checkbox menu
    Return: list of selected objects
    """
    names = list()
    checkbox_objects = list()
    for key in data:
        checkbox_objects.append(widgets.Checkbox(value=True, description=key))
        names.append(key)

    # generate dictionary of all arguments in the checkbox menu
    arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

    # divide the options in 4 columns, and generate horizontal grid
    chunk = int(round(len(checkbox_objects)/4))
    ui = widgets.HBox(
        [
            widgets.VBox(checkbox_objects[i:i+chunk]) for i in range(0, len(checkbox_objects), chunk)
        ]
    )

    # dynamically allocate values to variable
    selected_data = []
    def select_data(**kwargs):
        selected_data.clear()
        for key in kwargs:
            if kwargs[key] is True:
                selected_data.append(key)
        print(selected_data)

    out = widgets.interactive_output(select_data, arg_dict)
    display(ui)
    return selected_data

def get_years(data_path: str, year_options: dict = {}) -> dict:
    """
    Retrieve dataset available years.
    Input:
        data_path (str): string with directory where data resides.
    Return: dict of available years
    """
    for y in glob(f'{DATA_PATH}/*'):
        year = y.split('/')[-1]
        year_options[year] = year
    return sorted(year_options)

## 3.2 Specify data directory, masks and VIs

This notebook allows the user to select between Landsat (L30) and Sentinel-2 (S30) imagery in order to calculate the respective Masks. In this section the user will define the directory where the data resides, together with the years in question. In addition, the user will select the satellite for this run, including the desired Masks and VIs.

- **DATA_PATH (string):** define the directory where data resides, year directories should be located under this path followed by .hdf files. An example of this would be: /att/nobackup/user/L30, where /att/nobackup/user/L30/yyyy/imagery.hdf exists.
- **OUTPUT_PATH (string):** define the directory where data will be output, year directories will be auto-generated under this path followed by .hdf files. An example of this would be: /att/nobackup/user/output/L30, where /att/nobackup/user/output/L30/yyyy/imagery_vis.tif will be created.
- **YEARS (list string):** define the years that will be processed by this script (ALL will include all available years)
- **Selected Satellite (drop down menu):** choose between L30 (Landsat) and S30 (Sentinel)
- **Selected Masks (drop down menu):** choose all masks to apply for the final product

In this section the user will need to define the DATA_PATH variable, and select between the 3 available checkbox menus for year, masks, and VIs.

In [5]:
DATA_PATH = '/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30
# DATA_PATH = '/Users/jacaraba/Desktop/development/ilab/nccs-remote-sensing-work/data/S30'
# DATA_PATH = '/Users/jacaraba/Desktop/development/ilab/nccs-remote-sensing-work/data/L30'
# OUTPUT_PATH = '/Users/jacaraba/Desktop/development/ilab/nccs-remote-sensing-work/output/L30'
OUTPUT_PATH = '/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp/L30'
SATELLITE = DATA_PATH.split('/')[-1]  # selecting between L30 and S30 based on the last directory of DATA_PATH
YEAR_OPTIONS = get_years(DATA_PATH)  # retrieving dataset available years

print(f"Selecting satellite {SATELLITE} and DATA_PATH {DATA_PATH}")

Selecting satellite L30 and DATA_PATH /Users/jacaraba/Desktop/development/ilab/nccs-remote-sensing-work/data/L30


In [6]:
# years checkbox menu
print("Selected Years:")
YEARS = checkbox_menu(YEAR_OPTIONS)

# mask checkbox menu
print("Selected Masks:")
MASKS = checkbox_menu(MASK_OPTIONS)

# vis checkbox menu
print(f"Selected VIs for {SATELLITE}:")
VIS = checkbox_menu(VIS_OPTIONS_L30) if SATELLITE == 'L30' else checkbox_menu(VIS_OPTIONS_S30)

Selected Years:


Selected Masks:


Selected VIs for L30:


## 4. Start Mask and VIs Calculations

In this section, data files are retrieved and indices are calculated. Make sure to specify the variables in the previous section to allow for a seamless calculation.

## 4.1 Get all imagery files

In this section we mine through the DATA_PATH to retrive all filenames that will be processed. No input from the user is required in this section.

In [7]:
def get_filenames(data_path: str, years_var: list = ['ALL'], f_ext: str = '.hdf'):
    """
    Retrieve filenames to calculate indices from.
    Input:
        data_path (str): string with directory where data resides.
        years_var (str list): list of years to work with.
        f_ext (str): imagery filename extensions.
    Return: list of filenames to process
    """
    filenames = list()
    if 'ALL' in years_var:  # iterate over all years under the main data path
        filenames = glob(f'{data_path}/**/*{f_ext}', recursive=True) + filenames
    else: # iterate over the years specified by the user
        for y in years_var:
            filenames = glob(f'{data_path}/{y}/*{f_ext}', recursive=True) + filenames
    assert len(filenames) > 0, f"No files were found in {data_path}."
    return filenames

In [8]:
filenames = get_filenames(data_path=DATA_PATH, years_var=YEARS)
print(f"Processing {len(filenames)} files...")

Processing 2 files...


## 4.2 Define function to decode QA mask band

Here we need to decode the QA Mask Band. Examples of how to do it are outlined below. No input from the user is required in this section.

In [9]:
def get_binary(z: int, width: int = 8) -> str:
    """
    Retrieve binary representation.
    Input:
        z (int): string with value to decode
        width (int): int identity (HLS is 8 bits)
    Return: binary representation for a single value in str format
    """
    if z < 0:
        return '0' * width
    else:
        return np.binary_repr(z, width=width)
    
def get_mask(z: str, width: int = 8, start_bit: int = 0, end_bit: int = 1) -> int:
    """
    Retrieve mask from binary representation.
    Input:
        z (str): string with binary representation
        width (int): int identity (HLS is 8 bits)
        start_bit (int): position of bit in string (starts with 0)
        end_bit (int): position of bit in string (starts with 1)
    Return: return pixel mask value
    """
    if z == '0' * width:
        return 0
    else: # 1 - str2num(code(7))
        return 1 - int(z[start_bit:end_bit])

# vectorize get_binary and get_mask functions
v_get_binary = np.vectorize(get_binary, doc='Vectorized `get_binary_repr`')
v_get_mask = np.vectorize(get_mask, doc='Vectorized `get_mask`')

In [10]:
def apply_masks(ds, qa, masks_list: list = ['CLOUD'], width: int = 8) -> dict:
    """
    Retrieve masks from HLS dataset.
    """
    # TODO: make this function a multi-processing function
    # to parallelize the process.
    
    mask_ds = dict()

    if 'ADJCLOUD' in masks_list or 'ALL' in masks_list:
        mask_ds['ADJCLOUD'] = v_get_mask(qa, width=width, start_bit=5, end_bit=6)
        ds = ds * mask_ds['ADJCLOUD']

    if 'AQ' in masks_list or 'ALL' in masks_list:
        mask_ds['AQ'] = v_get_mask(qa, width=width, start_bit=0, end_bit=2)
        ds = ds * mask_ds['AQ']

    if 'CIRRUS' in masks_list or 'ALL' in masks_list:
        mask_ds['CIRRUS'] = v_get_mask(qa, width=width, start_bit=7, end_bit=8)
        ds = ds * mask_ds['CIRRUS']
    
    if 'CLOUD' in masks_list or 'ALL' in masks_list:
        mask_ds['CLOUD'] = v_get_mask(qa, width=width, start_bit=6, end_bit=7)
        ds = ds * mask_ds['CLOUD']
    
    if 'CLOUDSHADOW' in masks_list or 'ALL' in masks_list:
        mask_ds['CLOUDSHADOW'] = v_get_mask(qa, width=width, start_bit=4, end_bit=5)
        ds = ds * mask_ds['CLOUDSHADOW']

    if 'SNOW' in masks_list or 'ALL' in masks_list:
        mask_ds['SNOW'] = v_get_mask(qa, width=width, start_bit=3, end_bit=4)
        ds = ds * mask_ds['SNOW']

    if 'WATER' in masks_list or 'ALL' in masks_list:
        mask_ds['WATER'] = v_get_mask(qa, width=width, start_bit=2, end_bit=3)
        ds = ds * mask_ds['WATER']
    
    return ds, mask_ds

## fffff

In [11]:
def apply_vis_s30(ds, VIS_list: list = ['FCVI_VIS']):
    """
    S30 Bands - B01, B09, B10, B11, B12, QA, B02, B03, B04, B05, B06, B07, B08, B8A
    Common S30 and L30 VIs.
    """
    if 'CI705' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s./b05_s30_s)-1
        ds['CI705'] = (ds['B8A'] / ds['B05']) - 1.0

    if 'CI740' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s./b06_s30_s)-1
        ds['CI740'] = (ds['B8A'] / ds['B06']) - 1.0
    
    if 'CCCI' in VIS_list or 'ALL' in VIS_list:
        # ((b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s))./((b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s))
        ds['CCCI'] = ((ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])) / ((ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04']))

    if 'CIG' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s./b04_s30_s)-1
        ds['CIG'] = (ds['B8A'] / ds['B04']) - 1.0

    if 'CIRE' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s./b06_s30_s)-1, note: same formula as CI740
        ds['CIRE'] = (ds['B8A'] / ds['B06']) - 1.0

    if 'CVI' in VIS_list or 'ALL' in VIS_list:
        # ((b8a_s30_s).*(b04_s30_s))./((b03_s30_s).^2)
        ds['CVI'] = ((ds['B8A']) * (ds['B04'])) / ((ds['B03'])**2)

    if 'EVI' in VIS_list or 'ALL' in VIS_list:
        # 2.5.*(b8a_s30_s-b04_s30_s)./(b8a_s30_s+6*b04_s30_s-7.5*b02_s30_s+1)
        ds['EVI'] = 2.5 * (ds['B8A'] - ds['B04']) / (ds['B8A'] + 6 * ds['B04'] - 7.5 * ds['B02'] + 1)

    if 'FCVI' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-(b02_s30_s+b03_s30_s+b04_s30_s)./3)
        ds['FCVI'] = (ds['B8A'] - (ds['B02'] + ds['B03'] + ds['B04']) / 3.0)

    if 'FCVI_VIS' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-(b02_s30_s+b03_s30_s+b04_s30_s)./3)./((b02_s30_s+b03_s30_s+b04_s30_s)./3)
        ds['FCVI_VIS'] = (ds['B8A'] - (ds['B02'] + ds['B03'] + ds['B04']) / 3.0) / ((ds['B02'] + ds['B03'] + ds['B04']) / 3.0)
        
    if 'GLI' in VIS_list or 'ALL' in VIS_list:
        # (2*b03_s30_s-b04_s30_s-b02_s30_s)./(2*b03_s30_s+b04_s30_s+b02_s30_s)
        ds['GLI'] = (2 * ds['B03'] - ds['B04'] - ds['B02']) / (2 * ds['B03'] + ds['B04'] + ds['B02'])

    if 'GNDVI' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-b03_s30_s)./(b8a_s30_s+b03_s30_s)
        ds['GNDVI'] = (ds['B8A'] - ds['B03']) / (ds['B8A'] + ds['B03'])

    if 'MCARI' in VIS_list or 'ALL' in VIS_list:
        # ((b05_s30_s-b04_s30_s)-0.2.*(b05_s30_s-b03_s30_s))./((b06_s30_s)./(b04_s30_s));
        ds['MCARI'] = ((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B06'] / ds['B04'])

    if 'MCARI_MTVI2' in VIS_list or 'ALL' in VIS_list:
        # MCARI_S30./MTVI2_S30; dependent on MCARI and MTVI2
        ds['MCARI'] = ((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B06'] / ds['B04'])
        ds['MTVI2'] = 1.5 * ((1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03'])) / np.sqrt((2*ds['B8A']+1)**2 - (6 * ds['B8A'] - 5 * np.sqrt(ds['B04'])) - 0.5))
        ds['MCARI_MTVI2'] = ds['MCARI'] / ds['MTVI2']
    
    if 'MSAVI' in VIS_list or 'ALL' in VIS_list:
        # 0.5.*(2 * b8a_s30_s + 1 - sqrt( (2 * b8a_s30_s + 1).^2-8.*(b8a_s30_s-b04_s30_s) ) )
        ds['MSAVI'] = 0.5 * (2 * ds['B8A'] + 1 - np.sqrt( (2 * ds['B8A'] + 1)**2e-8 * (ds['B8A'] - ds['B04'])))
    
    if 'MTCI' in VIS_list or 'ALL' in VIS_list:
        # (b06_s30_s-b05_s30_s)./(b05_s30_s-b04_s30_s);
        ds['MTCI'] = (ds['B06'] - ds['B05']) / (ds['B05'] - ds['B04'])

    if 'MTVI1' in VIS_list or 'ALL' in VIS_list:
        # 1.2*(1.2*(b8a_s30_s-b03_s30_s)-2.5*(b04_s30_s-b03_s30_s))
        ds['MTVI1'] = 1.2 * (1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03']))

    if 'MTVI2' in VIS_list or 'ALL' in VIS_list:
        # 1.5*((1.2*(b8a_s30_s-b03_s30_s)-2.5.*(b04_s30_s-b03_s30_s))./sqrt((2*b8a_s30_s+1).^2-(6.*b8a_s30_s-5*sqrt(b04_s30_s))-0.5))
        ds['MTVI2'] = 1.5 * ((1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03'])) / np.sqrt((2*ds['B8A']+1)**2 - (6 * ds['B8A'] - 5 * np.sqrt(ds['B04'])) - 0.5))

    if 'NDREI' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s);
        ds['NDREI'] = (ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])

    if 'NDVI' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s)
        ds['NDVI'] = (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'])

    if 'NDVI705' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-b05_s30_s)./(b8a_s30_s+b05_s30_s);
        ds['NDVI705'] = (ds['B8A'] - ds['B05']) / (ds['B8A'] + ds['B05'])

    if 'NDVI740' in VIS_list or 'ALL' in VIS_list:
        # (b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s);
        ds['NDVI740'] = (ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])
 
    if 'NGRDI' in VIS_list or 'ALL' in VIS_list:
        # (b03_s30_s-b04_s30_s)./(b03_s30_s+b04_s30_s)
        ds['NGRDI'] = (ds['B03'] - ds['B04']) / (ds['B03'] + ds['B04'])

    if 'OSAVI' in VIS_list or 'ALL' in VIS_list:
        # (1+0.16).*(b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s+0.16)
        ds['OSAVI'] = (1 + 0.16) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.16)
    
    if 'RE1RE2' in VIS_list or 'ALL' in VIS_list:
        # (b06_s30_s./b05_s30_s)-1;
        ds['RE1RE2'] = (ds['B06'] / ds['B05']) - 1.0
    
    if 'REIP3' in VIS_list or 'ALL' in VIS_list:
        # 705+35.*(((((b04_s30_s+b8a_s30_s)./2)-b05_s30_s))./(b06_s30_s-b05_s30_s));
        ds['REIP3'] = 705 + 35 * (((((ds['B04'] + ds['B8A']) / 2) - ds['B05'])) / (ds['B06'] - ds['B05']))

    if 'SAVI' in VIS_list or 'ALL' in VIS_list:
        # (1+0.5).*(b8a_s30_s-b04_s30_s)./((b8a_s30_s+b04_s30_s+0.5))
        ds['SAVI'] = (1 + 0.5) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.5)
        
    if 'SR' in VIS_list or 'ALL' in VIS_list:
        # b8a_s30_s./b04_s30_s
        ds['SR'] = ds['B8A'] / ds['B04']

    if 'TCARI' in VIS_list or 'ALL' in VIS_list:
        # 3*(((b05_s30_s-b04_s30_s)-0.2*(b05_s30_s-b03_s30_s))./(b05_s30_s./b04_s30_s));
        ds['TCARI'] = 3 * (((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B05'] / ds['B04']))
    
    if 'TCARI_OSAVI' in VIS_list or 'ALL' in VIS_list:
        # TCARI_S30./OSAVI_S30; dependent on TCARI and OSAVI
        ds['TCARI'] = 3 * (((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B05'] / ds['B04']))
        ds['OSAVI'] = (1 + 0.16) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.16)
        ds['TCARI_OSAVI'] = ds['TCARI'] / ds['OSAVI']
    
    if 'TCI' in VIS_list or 'ALL' in VIS_list:
        # 1.2*(b05_s30_s-b03_s30_s)-1.5.*(b04_s30_s-b03_s30_s).*sqrt(b05_s30_s./b04_s30_s);
        ds['TCI'] = 1.2 * (ds['B05'] - ds['B03']) - 1.5 * (ds['B04'] - ds['B03']) * np.sqrt(ds['B05'] / ds['B04'])
    
    if 'TGI' in VIS_list or 'ALL' in VIS_list:
        # ((b04_s30_s-(b02_s30_s).*(b04_s30_s-b03_s30_s))-(b04_s30_s-(b03_s30_s).*(b04_s30_s-b02_s30_s))).*(-0.5);
        ds['TGI'] = ((ds['B04'] - (ds['B02']) * (ds['B04'] - ds['B03'])) - (ds['B04'] - (ds['B03']) * (ds['B04'] - ds['B02']))) * (-0.5)

    if 'TVI' in VIS_list or 'ALL' in VIS_list:
        # 0.5.*(120.*(b8a_s30_s-b03_s30_s)-200.*(b04_s30_s-b03_s30_s))
        ds['TVI'] = 0.5 * (120 * (ds['B8A'] - ds['B03']) - 200 * (ds['B04'] - ds['B03']))

    if 'VARI' in VIS_list or 'ALL' in VIS_list:
        # (b03_s30_s-b04_s30_s)./(b03_s30_s+b04_s30_s-b02_s30_s)
        ds['VARI'] = (ds['B03'] - ds['B04']) / (ds['B03'] + ds['B04'] - ds['B02'])

    return ds.fillna(0)

## fffff

In [12]:
def apply_vis_l30(ds, VIS_list: list = ['SR']):
    """
    L30 Bands - band01, band11, QA, band02, band03, band04, band05, band06, band07, band09, band10
    Common S30 and L30 VIs.
    """
    if 'CIG' in VIS_list or 'ALL' in VIS_list:
        # CIG_L30=((b05_l30_s)./(b04_l30_s))-1;
        ds['CIG'] = ((ds['band05']) / (ds['band04'])) - 1.0

    if 'CVI' in VIS_list or 'ALL' in VIS_list:
        # CVI_L30=(b05_l30_s).*((b04_l30_s)./((b03_l30_s).^2));
        ds['CVI'] = (ds['band05']) * ((ds['band04']) / ((ds['band03'])**2))
    
    if 'EVI' in VIS_list or 'ALL' in VIS_list:
        # EVI_L30=2.5.*(b05_l30_s-b04_l30_s)./(b05_l30_s+6.*(b04_l30_s)-7.5.*(b02_l30_s)+1);
        ds['EVI'] = 2.5 * (ds['band05'] - ds['band04']) / (ds['band05'] + 6 * (ds['band04']) - 7.5 * (ds['band02']) + 1)

    if 'FCVI' in VIS_list or 'ALL' in VIS_list:
        # FCVI_L30=(b05_l30_s-(b02_l30_s+b03_l30_s+b04_l30_s)./3);
        ds['FCVI'] = (ds['band05'] - (ds['band02'] + ds['band03'] + ds['band04']) / 3.0)
    
    if 'FCVI_VIS' in VIS_list or 'ALL' in VIS_list:
        # FCVI_VIS_L30=(b05_l30_s-(b02_l30_s+b03_l30_s+b04_l30_s)./3)./((b02_l30_s+b03_l30_s+b04_l30_s)./3);
        ds['FCVI_VIS'] = (ds['band05'] - (ds['band02'] + ds['band03'] + ds['band04']) / 3) / ((ds['band02'] + ds['band03'] + ds['band04']) / 3)

    if 'GLI' in VIS_list or 'ALL' in VIS_list:
        # GLI_L30=(2.*(b03_l30_s)-(b04_l30_s+b02_l30_s))./(2.*(b03_l30_s)+b04_l30_s+b02_l30_s);
        ds['GLI'] = (2 * (ds['band03']) - (ds['band04'] + ds['band02'])) / (2 * (ds['band03']) + ds['band04'] + ds['band02'])

    if 'GNDVI' in VIS_list or 'ALL' in VIS_list:
        # GNDVI_L30=(b05_l30_s-b03_l30_s)./(b05_l30_s+b03_l30_s);
        ds['GNDVI'] = (ds['band05'] - ds['band03']) / (ds['band05'] + ds['band03'])

    if 'MSAVI' in VIS_list or 'ALL' in VIS_list:
        # MSAVI_L30=0.5.*(2.*b05_l30_s+1-sqrt((2.*b05_l30_s+1).^2-8.*(b05_l30_s-b04_l30_s)));
        ds['MSAVI'] = 0.5 * (2 * ds['band05'] + 1 - np.sqrt((2 * ds['band05'] + 1)**2e-8 * (ds['band05'] - ds['band04'])))

    if 'MTVI1' in VIS_list or 'ALL' in VIS_list:
        # MTVI1_L30=1.2*(1.2*(b05_l30_s-b03_l30_s)-2.5*(b04_l30_s-b03_l30_s));
        ds['MTVI1'] = 1.2 * (1.2 * (ds['band05'] - ds['band03']) - 2.5 * (ds['band04'] - ds['band03']))
    
    # is the exponent after the negative? or is that a substraction?
    if 'MTVI2' in VIS_list or 'ALL' in VIS_list:
        # MTVI2_L30=1.5*((1.2*(b05_l30_s-b03_l30_s)-2.5.*(b04_l30_s-b03_l30_s))./sqrt((2*b05_l30_s+1).^2-(6.*b05_l30_s-5*sqrt(b04_l30_s))-0.5));
        ds['MTVI2'] = 1.5 * ((1.2 * (ds['band05'] - ds['band03']) - 2.5 * (ds['band04'] - ds['band03'])) / np.sqrt((2 * ds['band05'] + 1)**2 - (6 * ds['band05']- 5 * np.sqrt(ds['band04'])) - 0.5))
    
    if 'NDVI' in VIS_list or 'ALL' in VIS_list:
        # NDVI_L30=(b05_l30_s-b04_l30_s)./(b05_l30_s+b04_l30_s);
        ds['NDVI'] = (ds['band05'] - ds['band04']) / (ds['band05'] + ds['band04'])

    if 'NGRDI' in VIS_list or 'ALL' in VIS_list:
        # NGRDI_L30=(b03_l30_s-b04_l30_s)./(b03_l30_s+b04_l30_s);
        ds['NGRDI'] = (ds['band03'] - ds['band04']) / (ds['band03'] + ds['band04'])
    
    if 'OSAVI' in VIS_list or 'ALL' in VIS_list:
        # OSAVI_L30=(1+0.16).*(b05_l30_s-b04_l30_s)./(b05_l30_s+b04_l30_s+0.16);
        ds['OSAVI'] = (1 + 0.16) * (ds['band05'] - ds['band04']) / (ds['band05'] + ds['band04'] + 0.16)
    
    if 'SAVI' in VIS_list or 'ALL' in VIS_list:
        # SAVI_L30=(1+0.5).*(b05_l30_s-b04_l30_s)./((b05_l30_s+b04_l30_s+0.5)); 
        ds['SAVI'] = (1 + 0.5) * (ds['band05'] - ds['band04']) / ((ds['band05'] + ds['band04'] + 0.5))

    if 'SR' in VIS_list or 'ALL' in VIS_list:
        # SR_L30=b05_l30_s./b04_l30_s;
        ds['SR'] = ds['band05'] / ds['band04']

    if 'TVI' in VIS_list or 'ALL' in VIS_list:
        # TVI_L30=0.5.*(120.*(b05_l30_s-b03_l30_s)-200.*(b04_l30_s-b03_l30_s));
        ds['TVI'] = 0.5 * (120 * (ds['band05'] - ds['band03']) - 200 * (ds['band04'] - ds['band03']))

    if 'VARI' in VIS_list or 'ALL' in VIS_list:
        # VARI_L30=(b03_l30_s-b04_l30_s)./(b03_l30_s+b04_l30_s-b02_l30_s);
        ds['VARI'] = (ds['band03'] - ds['band04']) / (ds['band03'] + ds['band04'] - ds['band02'])
        
    return ds.fillna(0)

## Iterate over each file and output GeoTIF files

It is taking ~3 minutes to process each one of them. We can parallelize the process. Two things that can be parallelized here: process many files at the same time or parallelize functions to calculate indices. We are going to start parallelize the files processed at the same time.

In [13]:
%%time

# def process_filename(f: str, vis: list, masks: list, output_path: str):
def process_filename(f: str):
    
    # open the dataset from the hdf imagery
    hls_ds = xrx.open_rasterio(f)  #, chunks={'band': 1, 'x': 4096, 'y': 4096})

    # decode qa_mask for the generation of external masks
    qa_mask = v_get_binary(hls_ds['QA'].values)
    
    #print(hls_ds['band01'].min())
    
    # drop QA band and normalize
    hls_ds = hls_ds.drop('QA') / 10000.0
    
    # calculate indices
    hls_ds = apply_vis_l30(hls_ds, VIS) if SATELLITE == 'L30' else apply_vis_s30(hls_ds, VIS)

    #print(hls_ds['band01'].max())

    # Retrieve and apply required masks
    hls_ds, mask_ds = apply_masks(ds=hls_ds, qa=qa_mask, masks_list=MASKS)
    
    # create output directory if not present
    output_dir = os.path.join(OUTPUT_PATH, f.split('/')[-2])
    os.makedirs(output_dir, exist_ok=True)
    
    # define output filename
    output_file = os.path.join(output_dir, f.split('/')[-1][:-4] + '.gpp.vis.tif')
    
    # output raster to file
    hls_ds.isel(band=0).rio.to_raster(output_file)
    
    return output_file


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 5.96 µs


In [14]:
# %%time
# Serial approach
# for f in filenames:
#     process_filename(f, vis=VIS, masks=MASKS, output_path=OUTPUT_PATH)

In [15]:
# %%time
# parallel
#jobs = []
#for f in filenames:
#    p = mp.Process(target=process_filename, args=(f, VIS, MASKS, OUTPUT_PATH))
#    jobs.append(p)
#    p.start()
#    
#for j in jobs:
#    j.join()

def driver():
    PROCS = mp.cpu_count()
    with mp.Pool(PROCS) as pool:
        results = [pool.apply_async(process_filename, p) for p in filenames]
        
        for r in results:
            print(r.get())

#import multiprocessing
#pool = multiprocessing.Pool()
#results = pool.map(process_filename, filenames)
#pool.close()
#pool.join()
#print(results)

In [ ]:
driver()

In [ ]:
def f(x):
    return x*x
import numpy as np
from multiprocessing import Pool
p = Pool(processes=2)
result = p.map_async(f, np.arange(2))
print(result.get())

## Visual Validation

Validating masks visually

In [ ]:
# dropind =  ['band11', 'band04', 'band05', 'band06', 'band07', 'band09', 'band10']
# new_data = hls_ds.drop(dropind)
# type(new_data)

In [ ]:
# band01 = hls_ds['band01'].values
# band02 = hls_ds['band02'].values
# band03 = hls_ds['band03'].values
# nn = np.concatenate((band01, band02, band03), axis=0)
# nn = np.clip(nn, 0, 10000)
# nn = np.moveaxis(nn, 0, -1) / 10000.0

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(nn)
# plt.show()

In [ ]:
# from matplotlib import colors
# cmap = colors.ListedColormap(['r','b'])
# az = np.squeeze(hls_ds['SAVI'])
# plt.imshow(az)#, cmap=cmap)
# plt.show()

In [ ]:
# nn = np.concatenate((band01, band02, band03), axis=0)
# aw = nn * mask_ds['WATER']
# aw = np.moveaxis(aw, 0, -1) 
# plt.imshow(aw)
# plt.show()